In [4]:
#pip install nerus

In [9]:
#pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [1]:
from nerus import load_nerus
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn import tree
from sklearn.tree import export_graphviz
from IPython.display import Image
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn. preprocessing import OneHotEncoder
import collections
import re

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
docs = load_nerus('Downloads/nerus_lenta.conllu.gz')
docs = iter(docs)

In [3]:
docs

<generator object group_docs at 0x000001CA43618580>

In [4]:
doc = next(docs)

In [5]:
doc.id

'0'

In [6]:
sent = doc.sents[0]

In [7]:
df = pd.DataFrame()
df['X'] = []
df['Y'] = []
df['target'] = []

In [8]:
sent.morph.tokens[0].text

'Вице-премьер'

In [9]:
sent.morph.tokens[0].pos

'NOUN'

In [10]:
len(sent.morph.tokens)

23

In [11]:
len(doc.sents)

6

In [ ]:

doc = next(docs)
for i in range(len(doc.sents)):
    sent = doc.sents[i]
    for j in range(len(sent.morph.tokens)):
        row = {"X": sent.morph.tokens[j].text,"Y": sent.morph.tokens[j].pos, "target":sent.syntax.tokens[j].rel}
        df = df.append(row,ignore_index = True)

In [73]:
df = df.reset_index()

In [53]:
alphabet = ["а", "б", "в", "г", "д", "е", "ё", "ж", "з", 
"и", "й", "к", "л", "м", "н", "о", "п", "р",
"с", "т", "у", "ф", "х", "ц", "ч", "ш", "щ",
"ъ", "ы", "ь", "э", "ю", "я"]

In [ ]:
words = []
features = pd.DataFrame(columns = alphabet)
for i in range(df.shape[0]):
    words.append(df.X[i])
    features = features.append(dict(collections.Counter(df.X[i].lower())), ignore_index = True)

In [55]:
features = features.fillna(0)

In [59]:
features = features.drop("'", axis = 1)

In [75]:
features = features.reset_index()

In [76]:
features_prep = features.merge(df,'inner', on= 'index')

In [77]:
alphabet.append('X')

In [78]:
alphabet.append('target')

In [79]:
features_prep = features_prep[alphabet]

In [80]:
features_prep = features_prep[features_prep.drop(["X","target"],axis = 1).apply(sum,axis = 1) > 0]

In [81]:
features_prep.to_csv('nlp_dataset.csv',index = False)

In [82]:
df_final = pd.read_csv('nlp_dataset.csv')

In [170]:
features_prep['first'] = features_prep['X'].map(lambda x: x.lower()[0])
features_prep['last'] = features_prep['X'].map(lambda x: x.lower()[-1])
features_prep['len'] = features_prep['X'].map(lambda x: len(x))

In [136]:

df_final = features_prep[~(features_prep['target'].isin(['punct','list','csubj:pass']))]

In [138]:
X_train, X_test, y_train, y_test = train_test_split(df_final.drop(["target","X"],axis = 1), df_final.target, 
                                                    train_size=0.8, 
                                                    random_state=42,
                                                    stratify = df_final.target)

In [139]:
from sklearn.preprocessing import LabelEncoder

In [140]:
la_target = LabelEncoder()
first = LabelEncoder()
last = LabelEncoder()
y_train = la_target.fit_transform(y_train)
X_train['first'] = first.fit_transform(X_train['first'])
X_train['last'] = last.fit_transform(X_train['last'])

In [141]:
y_train

array([ 8, 22,  8, ..., 29,  3, 24])

#### Модель

In [142]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold


In [143]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))


In [160]:
parameters = {
    'min_child_weight': [1, 5, 10],
        'subsample': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
}

In [161]:
estimator = XGBClassifier(
    objective= 'binary:logistic',
    seed=42
)


In [162]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'roc_auc',
)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_estimator_

In [164]:
model_xgb = XGBClassifier(learning_rate=0.02, n_estimators=1000, objective='binary:logistic',
                    silent=True, nthread=1)
model_xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None, nthread=1,
              num_parallel_tree=None, ...)

In [165]:
y_test = la_target.transform(y_test)
X_test['first'] = first.transform(X_test['first'])
X_test['last'] = last.transform(X_test['last'])

In [167]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, model_xgb.predict(X_test))

0.6352040816326531

In [131]:
model_xgb.predict(X_test)

array([24,  8,  4, ..., 22,  8, 23], dtype=int64)

In [657]:
alphabet = ["а", "б", "в", "г", "д", "е", "ё", "ж", "з", 
"и", "й", "к", "л", "м", "н", "о", "п", "р",
"с", "т", "у", "ф", "х", "ц", "ч", "ш", "щ",
"ъ", "ы", "ь", "э", "ю", "я"]

In [658]:
random_word = pd.DataFrame(columns = alphabet)

In [662]:
random_word = random_word.append(dict(collections.Counter(str('колодец').lower())), ignore_index = True)
random_word = random_word[alphabet]

In [663]:
random_word = random_word.fillna(0)

In [664]:
la_target.inverse_transform(model_xgb.predict(random_word))

array(['nmod', 'nmod'], dtype=object)

In [166]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, model_xgb.predict(X_test))


0.6352040816326531

In [ ]:

problem = svm.problem(y, x) # создаем задачу
param = svm.parameter('-c 1 -s 4') # параметры обучения
model = svm.train(prob, param)

#### Как будет выглядеть

In [301]:


def remove_punctuation_and_english_letters(string):
    # Удаление знаков препинания
    cleaned_string = re.sub(r'[^\w\s]', '', string)
    
    # Удаление английских букв
    cleaned_string = re.sub(r'[a-zA-Z]', '', cleaned_string)
    
    return cleaned_string


word_dict = {
    'root': 'сказуемое',
    'amod': 'определение',
    'punct': 'знак препинания',
    'conj': 'союз',
    'nsubj': 'подлежащее',
    'obj': 'дополнение',
    'advmod': 'обстоятельство',
    'xcomp': 'сказуемое',
    'iobj': 'определение',
    'case': 'предлог',
    'obl': 'обстоятельство',
    'cc' : 'союз',
    'nmod': 'дополнение',
    'appos': 'дополнение',
    'det': 'определение',
    'cop': 'сказуемое',
    'csubj': 'сказуемое',
    'discourse': 'предлог',
    'mark': 'предлог',
    'parataxis': 'сказуемое',
    'flat:name': 'Имя, скорее всего дополнение или подлежащее',
    'acl': 'определение'
            }

### Пример

In [314]:
string = "Ты смотрел и слушал, когда другой мужчина кричал бы от ярости. Ты сдерживался, где другой бьёт."

In [315]:
string = remove_punctuation_and_english_letters(string)

# Разделение предложения на отдельные слова
words = string.split()

# Создание списка для индексов слов
word_indices = list(range(1, len(words) + 1))

# Создание пустого словаря для данных столбцов
data = {
    'first': [],
    'last': [],
    'len' :[]
}
for letter in 'йцукеёнгшщзхъфывапролджэячсмитьбю':
    data[letter] = []
# Заполнение данных столбцов
for word in words:
    data['first'].append(word.lower()[0])
    data['last'].append(word.lower()[-1])
    data['len'].append(len(word))
    for letter in 'йцукеёнгшщзхъфывапролджэячсмитьбю':
        data[letter].append(word.lower().count(letter))

# Создание датафрейма из словаря данных
df = pd.DataFrame(data, index=word_indices)
df
# Вывод датафрейма

df['first'] = first.transform(df['first'])
df['last']= last.transform(df['last'])

temp = la_target.inverse_transform(model_xgb.predict(df[X_train.columns.to_list()]))

In [316]:
for i in range(len(temp)):
    print(words[i]," : ",word_dict[temp[i]],'\n')

Ты  :  подлежащее 

смотрел  :  обстоятельство 

и  :  союз 

слушал  :  сказуемое 

когда  :  предлог 

другой  :  определение 

мужчина  :  подлежащее 

кричал  :  подлежащее 

бы  :  сказуемое 

от  :  предлог 

ярости  :  дополнение 

Ты  :  подлежащее 

сдерживался  :  дополнение 

где  :  обстоятельство 

другой  :  определение 

бьёт  :  дополнение 



In [310]:
#pip install natasha
# from natasha import (
#     Segmenter,
#     NewsSyntaxParser,
#     NewsEmbedding,
#     Doc
# )

# def syntax_func(text:str):
#     #Обработка языка
#     emb = NewsEmbedding()
#     segmenter = Segmenter()
#     syntax_parser = NewsSyntaxParser(emb)
#     doc = Doc(text)
#     doc.segment(segmenter)
#     doc.parse_syntax(syntax_parser)
#     word_list = [list(i) for i in doc.tokens]
#     word_output = []
#     for i in word_list:
#         try:
#             word_output += [str(i[2]) + '-' + str(word_dict[i[5]])]
#         except:
#             word_output += [str(i[2]) + '-' + str(i[5])]

#     return word_output

# syntax_func('фпьывм дыловтаж')